In [ ]:
# data imports
import numpy as np
import pandas as pd 
from pandas import Series,DataFrame

#math
import math

#plot imports
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style('darkgrid')
%matplotlib inline 

#machine learning imports 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

#for evaluating results
from sklearn import metrics

#import data
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
data = pd.read_excel('Donatelymess.xlsx', sheet_name='Sheet 1')

In [ ]:
data = data[['amount_raised','total_billed','Company name']].dropna()

In [ ]:
data1 = data[data.total_billed > 0]

In [ ]:
data1 = data1.set_index('Company name')

In [ ]:
data1.head()

In [ ]:
data1 = data1.rename(index= str, columns = {'amount_raised':'amount', 'total_billed':'billed'})

In [ ]:
data1 = data1.drop_duplicates(['amount'])

In [ ]:
data1.head()

In [ ]:
sns.pairplot(data1,kind='reg',size=10,x_vars=['amount'],y_vars=['billed'] )
plt.show

Check the causal relationship as a percent change - percent change between the amount raised and billed to client 

In [ ]:
lm1 = smf.ols('np.log(billed)~np.log(amount)', data = data1).fit()
lm1.summary()

Next check what the elastictiy treating amount raised on app as proxy for quantity and billed for price 

In [ ]:
lm2 = smf.ols('np.log(amount)~np.log(billed)', data = data1).fit()
lm2.summary()

In [ ]:
X = data1.amount

Y = data1.billed

In [ ]:
X = np.log(X)

Y = np.log(Y)

In [ ]:
X = X.reshape(-1,1)

Y = Y.reshape(-1,1)

In [ ]:
from sklearn.linear_model import LinearRegression

lreg = LinearRegression()

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y)

In [ ]:
linmodel = lreg.fit(X_train,Y_train)

accuracy = lreg.score(X_test,Y_test)

print(accuracy)

In [ ]:
classpredict = linmodel.predict(X_test)

print(metrics.r2_score(Y_test,classpredict))

In [ ]:
from sklearn import preprocessing

In [ ]:
datascaled = preprocessing.scale(data1)

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

In [ ]:
from sklearn.cluster import KMeans
sse = {}
K = range(1,11)
for k in K:
    kmeans = KMeans(n_clusters = k).fit(datascaled)
    data1["clusters"] = kmeans.labels_
    sse[k] = kmeans.inertia_
    
   
# Plot the elbow
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

In [ ]:
cluster = KMeans(n_clusters=3)

In [ ]:
cluster.fit(datascaled)

cluster.cluster_centers_

In [ ]:
data1[["clusters"]] = cluster.labels_

In [ ]:
data1.head()

In [ ]:
cluster2 = data1[data1.clusters == 2]

cluster1 = data1[data1.clusters == 1]

cluster0 = data1[data1.clusters == 0]

In [ ]:
writer = pd.ExcelWriter('donately_clusters_prime.xlsx')
cluster0.to_excel(writer,'Sheet1')
cluster1.to_excel(writer,'Sheet2')
cluster2.to_excel(writer,'Sheet3')
writer.save()

In [ ]:
cluster2.describe()

In [ ]:
cluster1.describe()

In [ ]:
cluster0.describe()

In [ ]:
sns.lmplot(x="amount", y="billed", hue="clusters", data=data1, markers=["o", "x","*"], fit_reg=False)